In [1]:
import os
import openai
import groq
import requests
import json
from dotenv import load_dotenv

In [4]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
personal_api_key = os.getenv("PERSONAL_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")



with open('context.md', 'r') as file:
    context = file.read()

In [5]:
# Initialize OpenAI client
client = openai.OpenAI(api_key=openai_api_key)

In [59]:
def encode_image_to_base64(image_path):
    import base64
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

#1 load the content of voice file to ai and translate to text and save to file
def transcribe_audio(audio_file_path: str):
    groq_client = groq.Groq(api_key=groq_api_key)
    response = groq_client.audio.transcriptions.create(
        file=(audio_file_path, open(audio_file_path, "rb")),
        language="pl",
        model="whisper-large-v3"
    )
    return response.text

def save_audio_link_to_file(link: str, out_path : str):
    audio_response = requests.get(link)
    with open(out_path, "wb") as file:
        file.write(audio_response.content)

In [64]:
task_url = f"https://centrala.ag3nts.org/data/{personal_api_key}/arxiv.txt"
task = requests.get(task_url)
questions = []
print("QUESTIONS:")
print(task.text)
for question in task.text.split("\n"):
    questions.append(question)
    print(question)

QUESTIONS:
01=jakiego owocu użyto podczas pierwszej próby transmisji materii w czasie?
02=Na rynku którego miasta wykonano testową fotografię użytą podczas testu przesyłania multimediów?
03=Co Bomba chciał znaleźć w Grudziądzu?
04=Resztki jakiego dania zostały pozostawione przez Rafała?
05=Od czego pochodzą litery BNW w nazwie nowego modelu językowego?

01=jakiego owocu użyto podczas pierwszej próby transmisji materii w czasie?
02=Na rynku którego miasta wykonano testową fotografię użytą podczas testu przesyłania multimediów?
03=Co Bomba chciał znaleźć w Grudziądzu?
04=Resztki jakiego dania zostały pozostawione przez Rafała?
05=Od czego pochodzą litery BNW w nazwie nowego modelu językowego?



In [26]:
import requests

response_req = requests.get("https://centrala.ag3nts.org/dane/arxiv-draft.html")
html_content = response_req.text

In [30]:
if not os.path.exists("article.md"):
    with open("article.md", "w") as file:
        file.write(html_content)

In [35]:
if not os.path.exists("article.json"):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[  
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": f"return all picutes and audio link included in the data below with short description of the content of the picture or audio return in json format with keys link, description {article_md}"
            }
        ],
        max_tokens=3000
    )
    article_json = json.loads(response.choices[0].message.content.replace("```json", "").replace("```", ""))
    with open("article.json", "w") as file:
        file.write(article_json)
    print(article_json)


{
  "media": [
    {
      "link": "https://centrala.ag3nts.org/dane/i/rynek.png",
      "description": "Original photograph capturing a sunset view from the market towards a church."
    },
    {
      "link": "https://centrala.ag3nts.org/dane/i/rynek_glitch.png",
      "description": "Recreated image showing slight damage and artifacts in the transmitted photo of the sunset."
    },
    {
      "link": "https://centrala.ag3nts.org/dane/i/fruit01.png",
      "description": "Image of the fruit before temporal transport."
    },
    {
      "link": "https://centrala.ag3nts.org/dane/i/fruit02.png",
      "description": "Image of the fruit after temporal transport, now missing its seeds."
    },
    {
      "link": "https://centrala.ag3nts.org/dane/i/strangefruit.png",
      "description": "An unexpected merged fruit resulting from the transmission of two different fruits."
    },
    {
      "link": "https://centrala.ag3nts.org/dane/i/resztki.png",
      "description": "Remnants of food

In [60]:
for item in article_json["media"]:   
    if item["link"].endswith(".png") and not os.path.exists(f"./media_desc/{item['link'].split('/')[-1]}.txt"):
        print(item["link"])
        # Create message with image
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert in image analysis. You are given an image and you need to respond with description of the content of the picture. Look at the given context of the picture and describe it in detail."
                },
                {
                    "role": "user",
                    "content": [
                        {
                        "type": "image_url",
                        "image_url": {
                            "url": item["link"]
                        }
                    },
                    {
                        "type": "text",     
                        "text": f"context of the picture {item['description']}"
                    }
                ]
            }
        ],
        max_tokens=3000
        )   
        print(response.choices[0].message.content)
        with open(f"media_desc/{item['link'].split('/')[-1]}.txt", "w") as file:
            file.write(response.choices[0].message.content)
    elif item["link"].endswith(".mp3") and not os.path.exists(f"./media_desc/{item['link'].split('/')[-1]}.txt"):
        if not os.path.exists("./media/audio.mp3"):
            save_audio_link_to_file(item["link"], "./media/audio.mp3")
        audio_text = transcribe_audio("./media/audio.mp3")    
        print(audio_text)
        with open(f"media_desc/{item['link'].split('/')[-1]}.txt", "w") as file:
            file.write(audio_text)
    else:
        print(f"picutre {item['link']} already exists")

picutre https://centrala.ag3nts.org/dane/i/rynek.png already exists
picutre https://centrala.ag3nts.org/dane/i/rynek_glitch.png already exists
picutre https://centrala.ag3nts.org/dane/i/fruit01.png already exists
picutre https://centrala.ag3nts.org/dane/i/fruit02.png already exists
picutre https://centrala.ag3nts.org/dane/i/strangefruit.png already exists
picutre https://centrala.ag3nts.org/dane/i/resztki.png already exists
 No i co teraz? No to teraz mi już nie powstrzymacie, jesteśmy tutaj sami. I trzeba tylko wykonać plan. Jak ja się do tego w ogóle zmieszczę? W sumie dobrze by zaplanowałem powszerzenie. Adam miał rację. Jedna informacja powinna się na w czasie. Jedna informacja. Dwa lata wcześniej posunie całe badania do przodu i wtedy już będzie z górki. Tak, i na odwagę. Z truskawką mowa nie wychło. Ale z ludźmi, no to czy... Dobra, jedna myfa, mowa pewnie wywoła. Ale Adam mówi, że to jest stabilne. Że to się wszystko uda. Trzeba tylko cofnąć się w czasie jeden i jedyny raz. Do G

In [61]:
with open("article.md", "r") as file:
    article_md = file.read()
    for item in article_json["media"]:
        if item["link"].endswith(".png") or item["link"].endswith(".mp3"):
            with open(f"media_desc/{item['link'].split('/')[-1]}.txt", "r") as f:
                article_md = article_md.replace(item["link"], f.read())
with open("article_with_desc.md", "w") as file:
    file.write(article_md)  

In [77]:
print(questions)

['01=jakiego owocu użyto podczas pierwszej próby transmisji materii w czasie?', '02=Na rynku którego miasta wykonano testową fotografię użytą podczas testu przesyłania multimediów?', '03=Co Bomba chciał znaleźć w Grudziądzu?', '04=Resztki jakiego dania zostały pozostawione przez Rafała?', '05=Od czego pochodzą litery BNW w nazwie nowego modelu językowego?', '']


In [78]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system", # Assistant role is for responses from the AI assistant
            "content": context   # System role is for defining core behavior/instructions
        },
        
        {
            "role": "assistant",
            "content": f"{article_md}"
        },
        {
            "role": "user",
            "content": task.text
        }
    ],
    max_tokens=3000
)
print(response.choices[0].message.content)

01=Podczas pierwszej próby transmisji materii w czasie użyto owocu truskawki.

02=Testową fotografię użytą podczas testu przesyłania multimediów wykonano na rynku Krakowa.

03=Bomba chciał znaleźć hotel w Grudziądzu.

04=Rafał pozostawił resztki pizzy na stole przy maszynie.

05=Litery BNW w nazwie nowego modelu językowego pochodzą od „Brave New World” (Nowy Wspaniały Świat).


In [91]:
response_sep_lines = response.choices[0].message.content.split("\n\n")
answers = {}
for lines in response_sep_lines:
    response_sep_id, response_sep_answer = lines.split("=")
    answers[response_sep_id] = response_sep_answer
print(answers)

{'01': 'Podczas pierwszej próby transmisji materii w czasie użyto owocu truskawki.', '02': 'Testową fotografię użytą podczas testu przesyłania multimediów wykonano na rynku Krakowa.', '03': 'Bomba chciał znaleźć hotel w Grudziądzu.', '04': 'Rafał pozostawił resztki pizzy na stole przy maszynie.', '05': 'Litery BNW w nazwie nowego modelu językowego pochodzą od „Brave New World” (Nowy Wspaniały Świat).'}


In [94]:

with open("answers.json", "w") as file:
    file.write(json.dumps(answers).encode('utf-8').decode('unicode_escape'))

In [95]:
answer_json = {
    "task": "arxiv",
    "apikey": personal_api_key,
    "answer": answers
}

In [96]:
answer_url = "https://centrala.ag3nts.org/report"
answer_response = requests.post(answer_url, json=answer_json)
print(answer_response.request.body)  # Displays the request body

# Assuming `login_response` is the response object from the login request
print("Status Code:", answer_response.status_code)  # Displays the status code (e.g., 200)
print("Response Body:", answer_response.text.encode('utf-8').decode('unicode-escape'))       # Displays the response content as a string

b'{"task": "arxiv", "apikey": "1400cbf0-b7dd-49ab-9342-6ad8fd26ba69", "answer": {"01": "Podczas pierwszej pr\\u00f3by transmisji materii w czasie u\\u017cyto owocu truskawki.", "02": "Testow\\u0105 fotografi\\u0119 u\\u017cyt\\u0105 podczas testu przesy\\u0142ania multimedi\\u00f3w wykonano na rynku Krakowa.", "03": "Bomba chcia\\u0142 znale\\u017a\\u0107 hotel w Grudzi\\u0105dzu.", "04": "Rafa\\u0142 pozostawi\\u0142 resztki pizzy na stole przy maszynie.", "05": "Litery BNW w nazwie nowego modelu j\\u0119zykowego pochodz\\u0105 od \\u201eBrave New World\\u201d (Nowy Wspania\\u0142y \\u015awiat)."}}'
Status Code: 200
Response Body: {
    "code": 0,
    "message": "{{FLG:BADANIA}}"
}
